# Lumen Data Science

PyTissue

Update or install important libraries

In [44]:
if False:
    !pip install -Uqq fastai
    !pip install -Uqq librosa

In [ ]:
from fastai.data.all import *
from fastai.vision.all import *
import librosa
import re
import itertools
import matplotlib.pyplot as plot
sys.path.append('../')

## Label getters

In [2]:
path = Path("../Dataset/Dataset/IRMAS_Validation_Data/")
get_song_files = FileGetter(extensions='.wav', recurse=True)
files = get_song_files(path)
print(f"(#{len(files)})", files[:6])

(#2874) [Path('../Dataset/Dataset/IRMAS_Validation_Data/(02) dont kill the whale-1.wav'), Path('../Dataset/Dataset/IRMAS_Validation_Data/(02) dont kill the whale-11.wav'), Path('../Dataset/Dataset/IRMAS_Validation_Data/(02) dont kill the whale-12.wav'), Path('../Dataset/Dataset/IRMAS_Validation_Data/(02) dont kill the whale-13.wav'), Path('../Dataset/Dataset/IRMAS_Validation_Data/(02) dont kill the whale-14.wav'), Path('../Dataset/Dataset/IRMAS_Validation_Data/(02) dont kill the whale-15.wav')]


In [3]:
def is_IRMAS_train(pat: Path):
    return str(pat).find("IRMAS_Train_Data") != -1

def is_IRMAS_valid(pat: Path):
    return str(pat).find("IRMAS_Validation_Data") != -1
    
song_path = Path("../Dataset/Dataset/IRMAS_Validation_Data/(02) dont kill the whale-2.wav")
is_IRMAS_train(song_path), is_IRMAS_valid(song_path)

(False, True)

In [4]:
def get_IRMAS_train_label(pat: Path):
    r = re.search("\[[^(\[\])]+\]", pat.name)
    if r:
        return [r.group()[1:-1]]
    return []
song_path = Path("../Dataset/Dataset/IRMAS_Train_Data/cel/[cel][cla]0001__1.wav")
get_IRMAS_train_label(song_path)

['cel']

In [5]:
def get_IRMAS_valid_label(pat: Path):
    with open(os.path.splitext(str(pat))[0] + ".txt") as file:
        return file.read().split()
song_path = Path("../Dataset/Dataset/IRMAS_Validation_Data/(02) dont kill the whale-2.wav")
get_IRMAS_valid_label(song_path)

['gel', 'voi']

In [6]:
def get_label(pat: Path):
    if is_IRMAS_train(pat):
        return get_IRMAS_train_label(pat)
    if is_IRMAS_valid(pat):
        return get_IRMAS_valid_label(pat)
song_path = Path("../Dataset/Dataset/IRMAS_Validation_Data/(02) dont kill the whale-2.wav")
get_label(song_path)

['gel', 'voi']

## Get items

In [ ]:
n_fft = 1024
hop_length = 512
f_min = 20
f_max = 8000

In [ ]:
def get_spec(pat: Path):
    clip, sample_rate = librosa.load(pat, sr=None)
    clip = clip[:sample_rate*3]
    stft = librosa.stft(clip, n_fft=n_fft, hop_length=hop_length)
    return stft

NameError: name 'Path' is not defined

In [ ]:
def spec2mag(stft):
    stft_magnitude, _ = librosa.magphase(stft)
    stft_magnitude_db = librosa.amplitude_to_db(stft_magnitude)
    return stft_magnitude_db

In [ ]:
def get_magspec(pat: Path):
    return spec2mag(get_spec(pat))

In [ ]:
def plot_magspec(stft_magnitude_db):
    plot.title('Spectrogram of a wav file with piano music')
    plot.imshow(spec)
    plot.xlabel('Sample')
    plot.ylabel('Amplitude')
    
song_path = Path("../Dataset/Dataset/IRMAS_Validation_Data/(02) dont kill the whale-2.wav")
spec = get_magspec(song_path)
plot_magspec(spec)

## Data augmentation

Combine two or more music files

## Creating metrics

In [50]:
def acc(x, y):
    return 1 - (((x > 0.5).float() - y).abs()).float().mean()
acc(torch.tensor([[0.1, 0.9, 0.4], [0.2, 0.2, 0.6]]), torch.tensor([[1, 1, 0], [0, 0, 1]]))

tensor(0.8333)

## Building Dataloaders

In [11]:
sdbl = DataBlock(
    blocks = [ImageBlock(), MultiCategoryBlock],
    get_items = get_song_files,
    splitter = RandomSplitter(),
    get_x = get_magspec,
    get_y = get_label
)

In [12]:
dls = sdbl.dataloaders(path, bs=64)

In [13]:
print(len(dls.train.dataset))
print(len(dls.valid.dataset))
dls.train

2300
574


In [14]:
# sdbl.summary(path)
print(dls.vocab)
# dls.train.show_batch()

['cel', 'cla', 'flu', 'gac', 'gel', 'org', 'pia', 'sax', 'tru', 'vio', 'voi']


## Track GPU usage

In [15]:
torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.current_device()

(True, 1, 0)

In [16]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1070'

In [29]:
import gc
def report_gpu():
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    gc.collect()
    torch.cuda.empty_cache()

In [30]:
report_gpu()

Memory Usage:
Allocated: 0.3 GB
Cached:    3.3 GB


## Building the model

In [51]:
learn = vision_learner(dls, resnet18, metrics=acc)
learn.fine_tune(1)

epoch,train_loss,valid_loss,acc,time
0,0.912555,0.709785,0.703041,00:44


epoch,train_loss,valid_loss,acc,time
0,0.689541,0.558350,0.809313,01:32


In [56]:
def train(arch, accum=1, finetune=True, epochs=12, bs=64):
    _dls = DataBlock(
        blocks = [ImageBlock(), MultiCategoryBlock],
        get_items = get_song_files,
        splitter = RandomSplitter(),
        get_x = get_magspec,
        get_y = get_label
    ).dataloaders(path, bs=bs)
    cbs = GradientAccumulation(64) if accum else []
    learn = vision_learner(_dls, arch, metrics=acc, cbs=cbs).to_fp16()
    if finetune:
        learn.fine_tune(epochs, 0.01)
        # return learn.tta(dl=dls.test_dl(tst_files))
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs, 0.01)

In [58]:
train(resnet18, epochs=6)

epoch,train_loss,valid_loss,acc,time
0,0.745288,0.495231,0.824517,01:23


epoch,train_loss,valid_loss,acc,time
0,0.293523,0.274175,0.897371,01:12
1,0.249524,0.298222,0.892936,01:10
2,0.205513,0.223830,0.920177,01:15
3,0.153906,0.175196,0.935382,01:26
4,0.102890,0.162720,0.938074,01:17
5,0.068943,0.156001,0.939816,01:21


NameError: name 'tst_files' is not defined